# 📊 Credit Risk Prediction - EDA & Baseline Models

**Análise Exploratória de Dados e Modelos Baseline**

Este notebook demonstra as competências necessárias para o **Bootcamp Microsoft Data Scientist Azure**:

- 🔍 **Exploração e preparação de dados**
- 📈 **Análise estatística e visualizações**
- 🤖 **Baseline modeling e comparação de algoritmos**
- 📊 **Avaliação de modelos com múltiplas métricas**
- 🎯 **Feature engineering e seleção**

---




In [ ]:
# Imports e configurações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import xgboost as xgb

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Bibliotecas importadas com sucesso!")


## 1. 📥 Carregamento e Preparação dos Dados

Nesta seção vamos:
- Carregar o dataset de risco de crédito
- Realizar análise inicial da estrutura dos dados
- Identificar missing values e outliers


In [ ]:
# Função para criar dados sintéticos (para demonstração)
def create_synthetic_credit_data(n_samples=1000, random_state=42):
    """Cria dataset sintético para demonstração do Bootcamp Microsoft Azure"""
    np.random.seed(random_state)
    
    data = {
        'age': np.random.randint(18, 80, n_samples),
        'annual_income': np.random.lognormal(10, 1, n_samples),
        'loan_amount': np.random.lognormal(9, 1, n_samples),
        'credit_score': np.random.randint(300, 850, n_samples),
        'employment_length': np.random.randint(0, 25, n_samples),
        'debt_to_income': np.random.uniform(0, 1, n_samples),
        'num_credit_lines': np.random.randint(1, 20, n_samples),
        'education': np.random.choice(['High School', 'College', 'Graduate'], n_samples),
        'home_ownership': np.random.choice(['Rent', 'Own', 'Mortgage'], n_samples),
    }
    
    df = pd.DataFrame(data)
    
    # Criar variável target com lógica de negócio realista
    risk_score = (
        (df['credit_score'] < 600).astype(int) * 0.4 +
        (df['debt_to_income'] > 0.5).astype(int) * 0.3 +
        (df['annual_income'] < 30000).astype(int) * 0.2 +
        np.random.random(n_samples) * 0.1
    )
    
    df['default'] = (risk_score > 0.5).astype(int)
    
    return df

# Carregar ou criar dados
try:
    df = pd.read_csv('../data/credit_risk.csv')
    print("📊 Dataset carregado do arquivo")
except:
    print("⚠️  Criando dataset sintético para demonstração...")
    df = create_synthetic_credit_data(2000, 42)
    # Salvar para uso futuro
    import os
    os.makedirs('../data', exist_ok=True)
    df.to_csv('../data/credit_risk.csv', index=False)
    print("💾 Dataset sintético salvo em ../data/credit_risk.csv")

print(f"✅ Dataset carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")


In [ ]:
# Análise inicial dos dados
print("="*50)
print("📋 RESUMO DO DATASET")
print("="*50)
print(f"📊 Shape: {df.shape}")
print(f"🎯 Taxa de Default: {df['default'].mean():.2%}")
print(f"💰 Renda média: ${df['annual_income'].mean():,.0f}")
print(f"📈 Score de crédito médio: {df['credit_score'].mean():.0f}")

print("\n" + "="*50)
print("📊 INFORMAÇÕES GERAIS")
print("="*50)
display(df.info())

print("\n" + "="*50) 
print("📈 ESTATÍSTICAS DESCRITIVAS")
print("="*50)
display(df.describe())

print("\n" + "="*50)
print("🔍 PRIMEIRAS 5 LINHAS")
print("="*50)
display(df.head())


## 2. 📊 Análise Exploratória de Dados (EDA)

Análise detalhada das variáveis e suas relações com o target (default).


In [ ]:
# Visualizações da distribuição do target
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Distribuição do target
df['default'].value_counts().plot(kind='bar', ax=axes[0], color=['lightgreen', 'salmon'])
axes[0].set_title('📊 Distribuição da Variável Target')
axes[0].set_xlabel('Default (0=Não, 1=Sim)')
axes[0].set_ylabel('Frequência')
axes[0].tick_params(axis='x', rotation=0)

# Adicionar percentuais nas barras
for i, v in enumerate(df['default'].value_counts()):
    pct = v / len(df) * 100
    axes[0].text(i, v + 50, f'{v}\n({pct:.1f}%)', ha='center', va='bottom', fontweight='bold')

# Taxa de default por faixa de credit score
credit_bins = pd.cut(df['credit_score'], bins=5, labels=['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto'])
default_rate = df.groupby(credit_bins)['default'].mean()
default_rate.plot(kind='bar', ax=axes[1], color='skyblue')
axes[1].set_title('📈 Taxa de Default por Faixa de Credit Score')
axes[1].set_xlabel('Faixa de Credit Score')
axes[1].set_ylabel('Taxa de Default')
axes[1].tick_params(axis='x', rotation=45)

# Adicionar valores nas barras
for i, v in enumerate(default_rate):
    axes[1].text(i, v + 0.01, f'{v:.2%}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"🎯 Insights iniciais:")
print(f"   • Dataset está {'balanceado' if 0.4 <= df['default'].mean() <= 0.6 else 'desbalanceado'}")
print(f"   • Taxa geral de default: {df['default'].mean():.2%}")
print(f"   • Credit Score médio dos que fizeram default: {df[df['default']==1]['credit_score'].mean():.0f}")
print(f"   • Credit Score médio dos que NÃO fizeram default: {df[df['default']==0]['credit_score'].mean():.0f}")


## 3. 🤖 Baseline Models & Comparação

Implementação e comparação de múltiplos algoritmos - competência essencial do Bootcamp Microsoft Azure:

- **Logistic Regression** (baseline linear)
- **Random Forest** (ensemble method)
- **XGBoost** (gradient boosting)


In [ ]:
# Preparação dos dados para modelagem
def prepare_data_for_modeling(df):
    """Preprocessa dados seguindo boas práticas do Bootcamp Microsoft Azure"""
    
    df_processed = df.copy()
    
    # 1. Separar features e target
    X = df_processed.drop('default', axis=1)
    y = df_processed['default']
    
    # 2. Encoding de variáveis categóricas
    categorical_cols = X.select_dtypes(include=['object']).columns
    
    for col in categorical_cols:
        # One-hot encoding para poucas categorias
        if X[col].nunique() <= 5:
            dummies = pd.get_dummies(X[col], prefix=col, drop_first=True)
            X = pd.concat([X.drop(col, axis=1), dummies], axis=1)
        else:
            # Label encoding para muitas categorias
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
    
    # 3. Feature Engineering 
    # Loan to Income Ratio
    X['loan_to_income_ratio'] = X['loan_amount'] / (X['annual_income'] + 1)
    
    # Credit Score normalizado
    X['credit_score_norm'] = (X['credit_score'] - 300) / (850 - 300)
    
    # Indicador de alto risco
    X['high_risk_profile'] = ((X['debt_to_income'] > 0.5) & (X['credit_score'] < 600)).astype(int)
    
    return X, y

# Preparar dados
print("🔄 Preparando dados para modelagem...")
X, y = prepare_data_for_modeling(df)

print(f"✅ Preparação concluída:")
print(f"   • Features: {X.shape[1]}")
print(f"   • Samples: {X.shape[0]}")
print(f"   • Target distribution: {y.value_counts().to_dict()}")

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Split realizado:")
print(f"   • Treino: {X_train.shape[0]} samples")
print(f"   • Teste: {X_test.shape[0]} samples")


In [ ]:
# Treinamento e comparação de múltiplos modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Definir modelos para comparação
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

# Treinar e avaliar cada modelo
results = []

print("🏋️  Treinando modelos...")
print("="*60)

for name, model in models.items():
    print(f"\n🤖 Treinando {name}...")
    
    # Treinamento
    model.fit(X_train, y_train)
    
    # Predições
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    metrics = {
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'AUC': roc_auc_score(y_test, y_pred_proba)
    }
    
    results.append(metrics)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    
    print(f"   ✅ Accuracy: {metrics['Accuracy']:.4f}")
    print(f"   📈 AUC: {metrics['AUC']:.4f}")
    print(f"   🔄 CV AUC: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# Criar DataFrame com resultados
results_df = pd.DataFrame(results)
results_df = results_df.round(4)

print("\n" + "="*60)
print("📊 COMPARAÇÃO DE MODELOS")
print("="*60)
display(results_df.set_index('Model').sort_values('AUC', ascending=False))

# Melhor modelo
best_model_name = results_df.loc[results_df['AUC'].idxmax(), 'Model']
best_auc = results_df['AUC'].max()

print(f"\n🏆 MELHOR MODELO: {best_model_name}")
print(f"🎯 AUC Score: {best_auc:.4f}")

# Salvar melhor modelo para uso posterior
best_model = models[best_model_name]
import joblib
import os
os.makedirs('../outputs', exist_ok=True)
joblib.dump(best_model, f'../outputs/{best_model_name.lower().replace(" ", "_")}_model.pkl')
print(f"💾 Modelo salvo: ../outputs/{best_model_name.lower().replace(' ', '_')}_model.pkl")
